In [ ]:
# Initial imports
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

global prices_df
global nn_model
global nn_future_predictions

In [ ]:
print("in neural network")

In [ ]:
# function to create sequences of data:
# the x values (input) will be the current closing price 
# the y values (output) will be the next day's closing price
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length), 0]
        y = data[i+seq_length, 0]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [ ]:
# extract closing prices
close = prices_df['Close'].values.reshape(-1,1)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_close = scaler.fit_transform(close)


In [ ]:
# create a sequence based on number of days:
seq_length = 60
X,y = create_sequences(scaled_close,seq_length)

In [ ]:
#split into training and testing data
#(80% of data for training, %20 for testing)
split = int(len(X) * .80)
X_train,X_test,y_train,y_test = X[:split],X[split:],y[:split],y[split:]

In [ ]:
# Reshape data for LSTM input [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Define the LSTM model with 50 neurons
nn_model = Sequential()
nn_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
nn_model.add(Dense(units=1))

In [ ]:
# Compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Print the model summary
print(nn_model.summary())

# Train and Evaluate

In [ ]:
# Train the model
nn_model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
# Evaluate the model using the testing data
nn_train_loss = nn_model.evaluate(X_train, y_train, verbose=0)
nn_test_loss = nn_model.evaluate(X_test, y_test, verbose=0)
print(f'Train Loss: {nn_train_loss:.4f}')
print(f'Test Loss: {nn_test_loss:.4f}')

# Predict using Historic Data (Backtest)

In [ ]:
# Predictions
y_pred = nn_model.predict(X_test)

In [ ]:
# Inverse transform the predictions (to get actual prices)
y_pred = scaler.inverse_transform(y_pred)

In [ ]:
# Inverse transform the actual values for comparison
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Compare predictions vs actual values
# Example: print the last 10 predictions and actual values
for i in range(10):
    print(f'Predicted: {y_pred[-(10-i)][0]:.2f}, Actual: {y_test_inv[-(10-i)][0]:.2f}')

In [ ]:
# jill - make the above a graph?

# Predict the Future

In [ ]:
#recall seq_length = 60, set above
# predict for 30 days
X_future = []
start_index = len(scaled_close) - seq_length
for i in range(start_index, start_index + 30):
    seq = scaled_close[i-seq_length:i,0]
    X_future.append(seq)

In [ ]:

# Convert X_future to numpy array and reshape for LSTM input [samples, time steps, features]
X_future = np.array(X_future)
X_future = np.reshape(X_future, (X_future.shape[0], X_future.shape[1], 1))

In [ ]:
# Predict future prices
nn_future_predictions = nn_model.predict(X_future)

#Inverse transform the predictions to get actual prices
nn_future_predictions = scaler.inverse_transform(nn_future_predictions)

In [ ]:
# graph future predictions

# derive future dates
last_date = prices_df.index[-1]

#add 1 day, then derive next 30 days
next_30_days = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=30)

# Convert to an array
nn_dates_future = next_30_days.values


In [ ]:
print("nn_future_predictions:")

In [ ]:
print(nn_future_predictions)

In [ ]:
# Plotting historical and predicted prices
plt.figure(figsize=(12, 6))
#plt.plot(prices_df['Date'], prices_df['Close'], label='Historical Prices')
plt.plot(prices_df['Close'], label='Historical Prices')
plt.plot(nn_dates_future, nn_future_predictions, label='Predicted Prices', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Historical and Predicted Stock Prices using LSTM Neural Network')
plt.legend()
plt.savefig('nn_predict.png')
plt.show()